# Simulation of ideal basic separations in gas chromatography with negative temperature gradients
## Supplementary material
_Jan Leppert, Leonid M. Blumberg, Matthias Wüst, Peter Boeker_


## How to use this Notebook

A Jupyter Notebook is an open-source interactive programming interface. It integrates code and its output together with text. Here the notebook is used togeteher with Binder (mybinder.org) to create a computing inviroment to run this notebook without the need to install additional software by the user. In this way the simulation used for the paper **"Simulation of ideal basic separations in gas chromatography with negative temperature gradients"** is shared as supplemental material and can be run by anyone.

Code, written in Julia, is typed in cells, labeled with "In []:". This code can be execuded by clicking the arrow in front of the cell, or by clicking on the cell and pressing shift+enter. While the code is executed, the label will change to "In [ * ]:". After it is executed the label gets a number, "In [1]:" and the result of the code be shown beneath it, labeled with "Out[1]:" (the same number as the code giving this result).

In some of the resulting output cells interactive elements will appear, e.g. sliders to choose a value or input fields to write a value. Changing of these should change the output, e.g. graphs, in the same output cell, without re-running the input cell. But other input cells, which use these parameters need to be re-run manually to update their output.  

Under the option "Cell" -> "Run All" the whole notebook can be execuded. 

In [39]:
#--------------------------------------------------------------
# Loading of packages and functions, this can take some time
using IBS
using Plots
using Interact
using LaTeXStrings
using DataFrames
using DrWatson
include("nbfunctions.jl");
#--------------------------------------------------------------

In [2]:
# test for interactivity, delete this later 
test = slider(1:10, label="test")
vbox(test, observe(test))

(div { style=Dict("display" => "flex","flex-direction" => "column") }
  Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["test"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 10,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Int64} with 1 listeners. Value:
5, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Jan\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Jan\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000003c1ff990, Task (runnable) @0x000000003c1ff990), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"14891924672771676585\",\"id\":\"706760423438757

### A. Definition of Parameters

In the following cell the parameters for the simulation of the ideal basic separation (IBS) with negative temperature gradients are set. With a toogle switch it can be decided to present the parameters dimensionless quantities or as quantities with SI-units.

The input parameters can be devided in three sub-catagories:
* system parameters ... describing the basic parameters of the IBS system
    * column length $L$ $\,\to\,$ number > 0 
    * hold-up time $t_M$ $\,\to\,$ number > 0 
    * plate height $H$ $\,\to\,$ number > 0  
    * pressure ratio $P$ $\,\to\,$ number $\geq$ 1, values > 1 result in a gradient of the mobile phase velocity due to gas decompression, limitation to $P\leq 1000$ (GC-MS).
* program parameters ... describing the parameters of the temperature program and the gradient
    * start temperature at column inlet $T_0$ $\,\to\,$ number > -9 (dimensionless) or -240°C (with units)
    * heating rate $r_T$ $\,\to\,$ a range$^{a,b}$, limitaion: 0.0001 to 10 (dimensionless) or 0.0003°C/$t_M$ to 300°C/$t_M$ (with units)
    * gradient $g_T$ $\,\to\,$ range$^a$ with negative values and zero must included, limitation: -10 to 0 (dimensionless) or -300°C/$L$ to 0°C/$L$ (with units)
* solute parameters ... describing the parameters of the solute triplett
    * characteristic temperature $T_{char}$ of the solute triplets $\,\to\,$ range$^a$, limitation: -2 to 8 (dimensionless) or -60°C to 240°C (with units)
    * difference of the characteristic temperature $\Delta T_{char}$ between the solutes of a pair $\,\to\,$ number > 0
    * initial zone width $\sigma_{init}$ $\,\to\,$ number $\geq$ 0, values > 0 result in non-ideal sample introduction.

*$^a$ a range consists of three values separated by ":" $\,\to\,$ $v_0:st:v_1$. The first value gives the start value $v_0$, the second value gives the step-wide $st$ and the third value is the end value $v_1$. From this range a vector is constructed $[ v_0, v_0 + st, ... , v_0 + n \cdot st, v_1]$. Alternatively the range can be defined by a vector of different values, e.g. $[1,2,4,6]$*

*$^b$ a range in the form of $10^{(v_0:st:v_1)}$ or $10^{[-1,-0.5,0]}=[10^{-1}, 10^{-0.5}, 10^{0}]$ is recommend.*

In [24]:
#--------------------------------
# Defining the Parameters
uipar=uiparameter()
#--------------------------------

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:attributes => Dict{Any,Any}(:type => "checkbox",Symbol("data-bind") => "checked: value, valueUpdate: 'change', event: {change: function (){this.changes(this.changes()+1)}}"),:id => "##583",:className => "is-checkradio ",:style => Dict{Any,Any}())), Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["quantities with SI-units"], Dict{Symbol,Any}(:attributes => Dict{SubString{String},SubString{String}}("className" => "","for" => "##583")))], Dict{Symbol,Any}(:className => "field interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"value" => (Observable{Bool} with 2 listeners. Value:
false, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Jan\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Jan\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"value" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"value\"]()) ? (this.valueFromJulia[\"value\"]=true, this.model[\"value\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000000159842f0, Task (runnable) @0x00000000159842f0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"3109414431206489111\",\"id\":\"11196004792873371447\",\"type\":\"observable\"}),\"value\":WebIO.getval({\"name\":\"value\",\"scope\":\"3109414431206489111\",\"id\":\"2430757601080833042\",\"type\":\"observable\"})};\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"3109414431206489111\",\"

In [20]:
#-----------------------------------------------------------------------------
# show the parameter values contrary to the parameter setting
uipar2 = uiparameter_conversion(uipar)
#-----------------------------------------------------------------------------

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any["parameters with units", Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any["system parameters:", Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}(:id => "container")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("value" => (Observable{String} with 1 listeners. Value:
"L = 15.0\\textrm{m}", nothing)), Set{String}(), nothing, Asset[Asset("js", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\katex.min.js"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\katex.min.css")], Dict{Any,Any}("value" => Any[WebIO.JSString("(function (txt){return this.k.render(txt,this.container)})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000000375e1430, Task (runnable) @0x00000000375e1430), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = ((function (k){this.k=k; this.container=this.dom.querySelector(\"#container\"); return k.render(\"L = 15.0\\\\textrm{m}\",this.container)}));\n    (WebIO.importBlock({\"data\":[{\"name\":null,\"type\":\"js\",\"url\":\"/assetserver/74e6785198a5a0f69a6f1eee20e61a71fdf3a67a-katex.min.js\"},{\"name\":null,\"type\":\"css\",\"url\":\"/assetserver/9ab1b73cf97531190bd101cc330c420a4cc664cd-katex.min.css\"}],\"type\":\"async_block\"})).then((imports) => handler.apply(this, imports));\n}\n")])], Dict{Symbol,Any}(:className => "interact-widget")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}(:id => "container")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("value" => (Observable{String} with 1 listeners. Value:
"t_M = 33.0\\textrm{s}", nothing)), Set{String}(), nothing, Asset[Asset("js", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\katex.min.js"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\katex.min.css")], Dict{Any,Any}("value" => Any[WebIO.JSString("(function (txt){return this.k.render(txt,this.container)})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000000375e1710, Task (runnable) @0x00000000375e1710), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = ((function (k){this.k=k; this.container=this.dom.querySelector(\"#container\"); return k.render(\"t_M = 33.0\\\\textrm{s}\",this.container)}));\n    (WebIO.importBlock({\"data\":[{\"name\":null,\"type\":\"js\",\"url\":\"/assetserver/74e6785198a5a0f69a6f1eee20e61a71fdf3a67a-katex.min.js\"},{\"name\":null,\"type\":\"css\",\"url\":\"/assetserver/9ab1b73cf97531190bd101cc330c420a4cc664cd-katex.min.css\"}],\"type\":\"async_block\"})).then((imports) => handler.apply(this, imports));\n}\n")])], Dict{Symbol,Any}(:className => "interact-widget")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}(:id => "container")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("value" => (Observable{String} with 1 listeners. Value:
"H = 0.0015\\textrm{m}", nothing)), Set{String}(), nothing, Asset[Asset("js", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\katex.min.js"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\katex.min.css")], Dict{Any,Any}("value" => Any[WebIO.JSString("(function (txt){return this.k.render(txt,this.container)})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Tas

### B. Simulation

After defining the parameters in a previous cell, the separations are simulated in the next cell for these parameters.

The simulation is discribed in the Supplementary materials, section S.2.  

Depending of the number of solute triplets, heating rates and gradients the simulation can take some time to be completted. 

In [26]:
#--------------------------
# run the simulation:
simres = simulation(uipar);
#--------------------------

calculation finished.


### C. Results
#### Resolution Ratio $\Xi (R_S)$

In [40]:
#-----------------------------------------------
# Resolution Ratios
ui1 = plot_resolution_ratio(simres)
#-----------------------------------------------

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:attributes => Dict{Any,Any}(:type => "checkbox",Symbol("data-bind") => "checked: value, valueUpdate: 'change', event: {change: function (){this.changes(this.changes()+1)}}"),:id => "##790",:className => "is-checkradio ",:style => Dict{Any,Any}())), Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["quantities with SI-unnits"], Dict{Symbol,Any}(:attributes => Dict{SubString{String},SubString{String}}("className" => "","for" => "##790")))], Dict{Symbol,Any}(:className => "field interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"value" => (Observable{Bool} with 2 listeners. Value:
false, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Jan\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Jan\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"value" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"value\"]()) ? (this.valueFromJulia[\"value\"]=true, this.model[\"value\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000039be45d0, Task (runnable) @0x0000000039be45d0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"12947221266089635100\",\"id\":\"13418295328102731474\",\"type\":\"observable\"}),\"value\":WebIO.getval({\"name\":\"value\",\"scope\":\"12947221266089635100\",\"id\":\"7119764657187262515\",\"type\":\"observable\"})};\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"12947221266089635100\

Figure 1 corresponds to Figure 3 of the paper.

Figure 1a) shows the resolution ratio $\Xi(R_S)$ over the thermal gradient for the selected heating rate (first slider) for the simulated solutes.

Figure 1b) shows the resolution ratio $\Xi(R_S)$ over the heating rate for the selected thermal gradient (second slider).

#### Chromatograms and performance ratios

In [41]:
#------------------------------------------------------------------------
# Plot of the simulation
uichrom = plot_chrom_prog(simres)
#------------------------------------------------------------------------

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:attributes => Dict{Any,Any}(:type => "checkbox",Symbol("data-bind") => "checked: value, valueUpdate: 'change', event: {change: function (){this.changes(this.changes()+1)}}"),:id => "##793",:className => "is-checkradio ",:style => Dict{Any,Any}())), Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["quantities with SI-unnits"], Dict{Symbol,Any}(:attributes => Dict{SubString{String},SubString{String}}("className" => "","for" => "##793")))], Dict{Symbol,Any}(:className => "field interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"value" => (Observable{Bool} with 2 listeners. Value:
false, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Jan\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Jan\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"value" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"value\"]()) ? (this.valueFromJulia[\"value\"]=true, this.model[\"value\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000003c1d3990, Task (runnable) @0x000000003c1d3990), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"1683392386690789641\",\"id\":\"2303299428493898588\",\"type\":\"observable\"}),\"value\":WebIO.getval({\"name\":\"value\",\"scope\":\"1683392386690789641\",\"id\":\"10374383186561053591\",\"type\":\"observable\"})};\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"1683392386690789641\",\

Figure 2a) shows the chromatogram of a defined solute triplet (by choosing the value of $T_{char}$ with a slider) for a certain heating rate ($r_T$-slider) and a certain gradient ($g_T$-slider). This figure corresponds to Figure 4b) of the paper.

Figure 2b) shows the coresponding temperature program is shown with the <font color=red>**temperatur at the inlet of the column in red**</font>, the <font color=blue>**temperature at the outlet of the column in blue**</font> and the **local temperatures of the three solutes in black**. This figure corresponds to Figure 2 of the paper.

Figure 2c) shows the ratio of the retention time difference of solute "a" and solute "b" $\Xi (\Delta t)$, the ratio of the peak widths $\Xi (\tau)$ and the ratio of the resolution $\Xi (R_S)$ over the different gradients $g_T$. This figure corresponds to Figure 4a) of the paper.

Figure 2d) shows the absolute value of the resolution $R_S$ over different gradients $g_T$. 

By varying the value of the gradient from values of zero to stronger gradients the influence of the gradient on the solute triplet is observable. With increasing strength of the gradient the peak width of the solutes is reduced, but also the difference of the retention times $\Delta t$ is reduced. The influence of the gradient on the retention time difference $\Delta t$ is stronger than the influence on the peak width and therfore the resulting separation, measured with the resolution $R_S$ declines with an increasing gradient.

If a non-IBS is simulated (e.g. non-ideal sample introduction $\sigma_{init}>0$, or mobile phase velocity gradient due to gas decompression $P>1$) cases exist, were the addition of a negative thermal gradient increase the resolution. The increased resolution will allways be smaller than the corresponding IBS-resolution ($\Xi(R_S)<1$).

#### Peaklist

In the following table the peak list for a choosen set of heating rate ($r_T$-slider, resp. $R_T$-slider) and gradient ($g_T$-slider resp. $G_T$-slider) is shown. The first column shows the characteristic temperature of the solutes, the second to fourth column show the retention times of the three solutes and the fifth to seventh column show the peak width of the three solutes. The eigth to tenth column shows the difference of the retention times, the mean peak width and the resolution $R_S$ between solutes "a" and "b". 

If the quantities are are expressed with units, the characteristic temperature is given in °C and the retention times and peak widths are given in second.

In [38]:
#-------------------------------------
# Peaklist
uipl=peaklist(simres)
#-------------------------------------

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:attributes => Dict{Any,Any}(:type => "checkbox",Symbol("data-bind") => "checked: value, valueUpdate: 'change', event: {change: function (){this.changes(this.changes()+1)}}"),:id => "##776",:className => "is-checkradio ",:style => Dict{Any,Any}())), Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["quantities with SI-unnits"], Dict{Symbol,Any}(:attributes => Dict{SubString{String},SubString{String}}("className" => "","for" => "##776")))], Dict{Symbol,Any}(:className => "field interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"value" => (Observable{Bool} with 2 listeners. Value:
false, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Jan\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Jan\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Jan\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"value" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"value\"]()) ? (this.valueFromJulia[\"value\"]=true, this.model[\"value\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000003ea0e570, Task (runnable) @0x000000003ea0e570), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"15801441381003518434\",\"id\":\"2605116704594384901\",\"type\":\"observable\"}),\"value\":WebIO.getval({\"name\":\"value\",\"scope\":\"15801441381003518434\",\"id\":\"4817165849660910229\",\"type\":\"observable\"})};\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"15801441381003518434\"